# GUI of CPlantBox
This is an CPlantBox with GUI, it can be run on Binder.

In [1]:
from jupyter_dash import JupyterDash
from jupyterlab_dash import AppViewer
import dash
import dash_core_components as dcc
import dash_html_components as html
import xml.etree.ElementTree as ET
import pandas as pd
from dash.dependencies import Input, Output, State
from CPlantBox_PiafMunch import *

In [2]:
p = pb.Plant()
# Step 2: read XML parameters
p.openXML('../../modelparameter/plant/small_2020.xml')

# try other leaf arrangements
# p.openXML('../../modelparameter/plant/leaf_opposite_decussate.xml')
# p.openXML('../../modelparameter/plant/leaf_alternate.xml')

# try a more complex plant
# p.openXML('../../modelparameter/plant/logo_plant.xml')

# initialize the parameters
# p.initialize()

# # simulation for 3 days
# p.simulate(3) 


In [3]:
help(p)

Help on Plant in module plantbox object:

class Plant(Organism)
 |  Method resolution order:
 |      Plant
 |      Organism
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(self: plantbox.Plant) -> None
 |  
 |  createGrowthFunction(...)
 |      createGrowthFunction(self: plantbox.Plant, arg0: int) -> CPlantBox::GrowthFunction
 |  
 |  createTropismFunction(...)
 |      createTropismFunction(self: plantbox.Plant, arg0: int, arg1: float, arg2: float) -> plantbox.Tropism
 |  
 |  getSeed(...)
 |      getSeed(self: plantbox.Plant) -> plantbox.Seed
 |  
 |  initCallbacks(...)
 |      initCallbacks(self: plantbox.Plant) -> None
 |  
 |  openXML(...)
 |      openXML(self: plantbox.Plant, arg0: str) -> None
 |  
 |  reset(...)
 |      reset(self: plantbox.Plant) -> None
 |  
 |  setGeometry(...)
 |      setGeometry(self: plantbox.Plant, arg0: plantbox.SignedDistanceFunction) -> None
 |  
 |  setTropism(...

In [4]:
params = [ p.getOrganRandomParameter(i) for i in range(1,5) ]

In [5]:
params

[[<plantbox.SeedRandomParameter at 0x7f114d3ee3b0>],

In [6]:
params[1] = params[1][1:]
params[2] = params[2][1:]
params[3] = params[3][1:]

In [7]:
params= sum(params, [])

In [ ]:
pp = pb.Plant()
# pp.setOrganRandomParameter(params[0])
pp.setOrganRandomParameter(params[3])

In [10]:
print(params[3])

[Parameters of mainstem]
Variable		Value		Deviation		Description
===
gf                	1		0			Growth function number [1]
lnf               	0		-			Type of inter-branching distance (0 homogeneous, 1 linear inc, 2 linear dec, 3 exp inc, 4 exp dec)
organType         	3		-			Organ type (unspecified organ = 0, seed = 1, root = 2, stem = 3, leaf = 4)
subType           	1		-			Unique identifier of this sub type
tropismT          	0		-			Type of stem tropism (plagio = 0, gravi = 1, exo = 2, hydro, chemo = 3)
BetaDev           	0		-			RevRotation deviation
InitBeta          	1		-			Initial RevRotation
RotBeta           	0		-			RevRotation of the stem
a                 	0.2		0.02			Stem radius [cm]
dx                	0.25		-			Axial resolution [cm] (maximal segment size)
la                	0		0			Apical zone [cm]
lb                	1		0			Basal zone [cm]
lmax              	4		0			Maximal stem length [cm]
ln                	1		0			Inter-lateral distance [cm]
r                 	2		0			Initial gro

In [ ]:
pp = pb.Plant()
for i,param in enumerate(params):
    pp.setOrganRandomParameter(param)
pp.initialize()
pp.simulate(20)


In [4]:
# Build App
# JupyterDash.infer_jupyter_proxy_config()
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server







def transform_value(value):
    return 10 ** value
BACKGROUND = 'rgb(255, 255, 255)'

app.scripts.config.serve_locally = False
app.title = 'CPlantBox' 
p = pb.Plant()
xmlname = 'FSPM2020.xml'
# ROOT_P = ()





app.layout = html.Div([html.Div([ html.Div([
            dcc.Dropdown(
        id='parameter_name_dropdown',
        options=[
            {'label': 'FSPM2020 example', 'value': 'FSPM2020'},
            {'label': 'Phloem', 'value': 'Phloem'},
            {'label': 'MAIZE', 'value': 'AMAIZE'}
        ],
        value='FSPM2020'
    ),
    html.Div([
    dcc.Dropdown(
        id='organ-dropdown',
        
        value='seed0'
    ),

    dcc.Dropdown(id='parameter-dropdown'),

    html.Div(id='display-selected-values'),
            dcc.Slider(
            id='slider-updatemode',
            marks={i: '{}'.format(10 ** i) for i in range(4)},
            max=3,
            value=2,
            step=0.01,
            updatemode='mouseup'
        ),
        html.Div(id='updatemode-output-container', style={'margin-top': 20}),
        dcc.Input(id='xml_ouput_name', type='text', value='Test_FSPM2020'),
        html.Button(id='save_XML', n_clicks=0, children='update_parameter'),

                
            ])           
            
        ], className='three columns'),]),
        

        
        
   
    
    
    html.Div([
        dcc.Slider(id='growing-days', min=1, max=60, value =5, 
                  marks={
                      0: '0 day',
                      5: '5 days',
                      30: '30 days',
                  }),
        html.Button('Run the model', id='run_model'),
        dcc.Loading(
                    id="loading-2",
                    children=[html.Div(dcc.Graph(id='3d-graph', style={'height' : '90vh'}),
                                               
                                      )],
                    type="circle",
        )],className="six columns")]
)
 


@app.callback(
    Output('parameter-dropdown', 'options'),
    [Input('organ-dropdown', 'value')])
def set_organ_options(selected_organ):
    return [{'label': i, 'value': i} for i in parameter_options[selected_organ]]


@app.callback(
    Output('parameter-dropdown', 'value'),
    [Input('parameter-dropdown', 'options')])
def set_organ_value(available_options):
    return available_options[0]['value']


@app.callback(
    Output('display-selected-values', 'children'),
    [Input('organ-dropdown', 'value'),
     Input('parameter-dropdown', 'value')])
def set_display_children(selected_organ, selected_parameter):
    parameter_element=root.find(".//*..[@type='{}'][@subType='{}']/*[@name='{}']".format(selected_organ[:-1] ,selected_organ[-1:], selected_parameter )).attrib
    pe_value=parameter_element['value']
    if 'dev' in parameter_element:
        pe_dev=parameter_element['dev']
    
    else:
        pe_dev='no deviation' 
        pe_value=parameter_element['value']  
    return u'value is {}, deviation is {}'.format(pe_value,pe_dev)

@app.callback(Output('slider-updatemode', 'value'),
              [Input('save_XML', 'n_clicks')],
              [State('organ-dropdown', 'value'),
               State('parameter-dropdown', 'value'),
               State('parameter_name_dropdown', 'value')])
def display_value(value,selected_organ, selected_parameter, name ):
    #tree = ET.parse("modelparameter/"+name+".xml")
    #root = tree.getroot()
    parameter_element=root.find(".//*..[@type='{}'][@subType='{}']/*[@name='{}']".format(selected_organ[:-1] ,selected_organ[-1:], selected_parameter )).attrib
    pe_value=parameter_element['value']
    if 'dev' in parameter_element:
        pe_dev=parameter_element['dev']    
    else:
        pe_dev='0' 
    root.find(".//*..[@type='{}'][@subType='{}']/*[@name='{}']".format(selected_organ[:-1] ,selected_organ[-1:], selected_parameter )).set('value',str(transform_value(value)))
    
    return float(pe_value)
	      

@app.callback(Output('organ-dropdown', 'options'), 
 [Input('parameter_name_dropdown', 'value')]
              
              )
def update_output(value):
    tree = ET.parse('../../modelparameter/plant/{}.xml'.format(value))
    root = tree.getroot()
    parameter_options={}
    parameter_order = 0
    for organ in root.iter('organ'): 
        list=[]
        for parameter in organ.iter('parameter'): 
        # print(parameter.attrib['name'])

            list.append(parameter.attrib['name']) 
            parameter_order = parameter_order+1
        parameter_options[organ.attrib['type']+organ.attrib['subType']] = list
    #print(organ.attrib)     
    return {'label' : parameter_options.keys()[0] ,'value' : parameter_options.keys()[0] }

	

	
@app.callback(
    Output('output-state2', 'children'),
    [Input('save_XML', 'n_clicks')],
[State('xml_ouput_name', 'value')]
)
def update_output2(n_clicks, value):
    tree.write('../../modelparameter/plant/'+value+".xml")
    xmlname = value+".xml"
    return 'saved in name:  {}'.format(value)

@app.callback(Output('updatemode-output-container', 'children'),
              [Input('slider-updatemode', 'value')],
              [State('organ-dropdown', 'value'),
               State('parameter-dropdown', 'value'),
               State('parameter_name_dropdown', 'value')])
def display_value(value,selected_organ, selected_parameter, name ):
    #tree = ET.parse("modelparameter/"+name+".xml")
    #root = tree.getroot()
    parameter_element=root.find(".//*..[@type='{}'][@subType='{}']/*[@name='{}']".format(selected_organ[:-1] ,selected_organ[-1:], selected_parameter )).attrib
    pe_value=parameter_element['value']
    if 'dev' in parameter_element:
        pe_dev=parameter_element['dev']    
    else:
        pe_dev='0' 
    root.find(".//*..[@type='{}'][@subType='{}']/*[@name='{}']".format(selected_organ[:-1] ,selected_organ[-1:], selected_parameter )).set('value',str(transform_value(value)))
    return 'Linear Value: {} | \
            Log Value: {:0.2f} original value {:f} and dev {:f}'.format(value, transform_value(value),float(pe_value),float(pe_dev))
    





@app.callback(
    Output('3d-graph', 'figure'),
    [Input('run_model', 'n_clicks')],
    [ State('growing-days', 'value')])
def update_figure(n_clicks1,  time):
    
    full_path = '../../modelparameter/plant/{}'.format(xmlname)
    p.openXML(full_path)
    p.initialize()

    p.simulate(time)

    fig = visual_plant_l(p)
    return fig


In [5]:
# app.run_server()
app.run_server(debug = True)


Dash app running on http://127.0.0.1:8050/


In [6]:
tree = ET.parse('../../modelparameter/plant/{}.xml'.format('FSPM2020'))
root = tree.getroot()
parameter_options={}
parameter_order = 0
for organ in root.iter('organ'): 
    list=[]
    for parameter in organ.iter('parameter'): 
    # print(parameter.attrib['name'])

        list.append(parameter.attrib['name']) 
        parameter_order = parameter_order+1
    parameter_options[organ.attrib['type']+organ.attrib['subType']] = list

NameError: name 'parameter_option' is not defined